In [1]:
# Python standard modules
import sys
from os import mkdir
from os.path import join
from os.path import isdir
from os.path import isfile
from shutil import rmtree

# Community Packages
import numpy as np
import pandas as pd

# My packages
sys.path.insert(0, "../scs/")
from prepare_R_data import prepare_R_data
from data_loading import load_sn_data
import learn

sys.path.insert(0, "../scs/models")
import feed_forward

from importlib import reload

In [3]:
reload(learn)

dir_model = join(dir_models, "dev_model")
if isdir(dir_model):
    rmtree(dir_model)
    
if not isdir(dir_model):
    mkdir(dir_model)
    mkdir(join(dir_model, "backup"))
    mkdir(join(dir_model, "data"))
    mkdir(join(dir_model, "results"))

R = 100
hp = {
    "phase_range": (-20, 50),
    "ptp_range": (0.1, 100),
    "wvl_range": (4500, 7000),
    "train_frac": 0.50,
    "noise_scale": 0.1,
    "spike_scale": 1.0,
    "max_spikes": 3,
    "random_state": None,
    
    "model_type": "feed_forward",
    "add_dim": False,
    "swap": False,
    "units": [1024, 1024, 1024],
    "activation": "relu",
    "dropout": 0.1,
    
    # "model_type": "transformer_encoder",
    # "add_dim": True,
    # "swap": True,
    # "encoder_blocks": 1,
    # "encoder_heads": 8,
    # "encoder_key_dim": 32,
    # "encoder_proj_dim": 2048,
    # "encoder_dropout_attention": 0.1,
    # "encoder_dropout_projection": 0.1,
    # "feed_forward_units": [1024, 1024, 1024],
    # "feed_forward_activation": "relu",
    # "feed_forward_dropout": 0.1,

    "lr_schedule": "constant_lr",
    "lr0": 1e-5,
    "epochs": 20,
    "batch_size": 32,
}

learn.run_SCS(
    dir_model,
    R,
    hp,
    file_raw_data=file_raw_data,
)

Numpy random state: None
Reading in raw data file: /home/2649/repos/SCS/data/sn_data.parquet
Degrading the dataset to R = 100
Preprocessing the dataset.
Phase Range (in days): (-20, 50)
Peak-to-Peak Range (in spectral units): (0.1, 100)
Wavelength Range (in Angstroms): (4500, 7000)
Perform a special train-test split on the dataset.
This train-test split splits the dataset by SNe, not by spectra.
Fraction of SNe in the training set: 0.5
Augmenting the dataset.
Scale of noise to augment the data with: 0.1
Scale of the spikes to augment the data with: 1.0
Maximum spikes to augment the dataset with: 3
Data preparation complete.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 139)]             0         
                                                                 
 dense (Dense)               (None, 1024)              143360    
                       

2023-04-08 17:01:12.100905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/shared/slurm/lib:/lustre/lrspec/users/2649/anaconda3/envs/fox/lib/:/lustre/lrspec/users/2649/anaconda3/envs/fox/lib/
2023-04-08 17:01:12.100957: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-08 17:01:12.100998: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (r2e00): /proc/driver/nvidia/version does not exist
2023-04-08 17:01:12.101225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appro


Epoch 1: val_loss improved from inf to 1.37589, saving model to /lustre/lrspec/users/2649/models/dev_model/model.hdf5
553/553 - 6s - loss: 1.7177 - ca: 0.6445 - f1: 0.6357 - val_loss: 1.3759 - val_ca: 0.6226 - val_f1: 0.4349 - lr: 1.0000e-05 - 6s/epoch - 11ms/step
Epoch 2/20

Epoch 2: val_loss improved from 1.37589 to 0.99003, saving model to /lustre/lrspec/users/2649/models/dev_model/model.hdf5
553/553 - 5s - loss: 0.6297 - ca: 0.8484 - f1: 0.8454 - val_loss: 0.9900 - val_ca: 0.7140 - val_f1: 0.5221 - lr: 1.0000e-05 - 5s/epoch - 9ms/step
Epoch 3/20

Epoch 3: val_loss improved from 0.99003 to 0.90303, saving model to /lustre/lrspec/users/2649/models/dev_model/model.hdf5
553/553 - 5s - loss: 0.3974 - ca: 0.8901 - f1: 0.8900 - val_loss: 0.9030 - val_ca: 0.7247 - val_f1: 0.5350 - lr: 1.0000e-05 - 5s/epoch - 9ms/step
Epoch 4/20

Epoch 4: val_loss improved from 0.90303 to 0.80848, saving model to /lustre/lrspec/users/2649/models/dev_model/model.hdf5
553/553 - 5s - loss: 0.2971 - ca: 0.9144

<Figure size 814.587x576 with 0 Axes>

In [4]:
dir_model_backup = join(dir_model, "backup")
dir_model_data = join(dir_model, "data")
dir_model_results = join(dir_model, "results")

file_model = join(dir_model, "model.hdf5")
file_model_history = join(dir_model, "history.log")
file_model_results = join(dir_model_results, "results.json")
file_model_hp = join(dir_model, "hp.json")
file_model_curves = join(dir_model_results, "curves.pdf")

In [8]:
model = learn.load_SCS_model(file_model, file_model_hp)

In [21]:
import hp_sets

In [25]:
PG = hp_sets.dev9()

In [27]:
PG[10]

{'wvl_range': (4500, 7000),
 'train_frac': 0.8,
 'spike_scale': 1.045639552591273,
 'retry': 10,
 'random_state': 1415,
 'ptp_range': (0.1, 100),
 'phase_range': (-20, 50),
 'num_transformer_blocks': 0,
 'num_heads': 8,
 'num_feed_forward_layers': 3,
 'noise_scale': 0.15848931924611134,
 'max_spikes': 3,
 'lr_schedule': 'constant_lr',
 'lr0': 0.000727895384,
 'kr_l2': 0,
 'key_dim': 8,
 'initial_projection': 100,
 'filters': 512,
 'feed_forward_layer_size': 1024,
 'dropout_projection': 0.1,
 'dropout_feed_forward': 0.7,
 'dropout_attention': 0.1,
 'br_l2': 0,
 'ar_l2': 0}